In [1]:
import yfinance as yf
import pandas as pd

# 5 NIFTY 50 stocks from different sectors
stocks = ['RELIANCE.NS', 'HDFCBANK.NS', 'INFY.NS', 'TATAMOTORS.NS', 'HINDUNILVR.NS']

# Fetch historical daily data (last 60 days)
def fetch_stock_data(symbols, period='max', interval='1d'):
    data = {}
    for symbol in symbols:
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period=period, interval=interval)
        hist.reset_index(inplace=True)
        data[symbol] = hist
    return data

# Call the function
stock_data = fetch_stock_data(stocks)

# Example: Display first few rows of RELIANCE data
print("RELIANCE.NS Data:")
print(stock_data['RELIANCE.NS'].head())


RELIANCE.NS Data:
                       Date      Open      High       Low     Close  \
0 1996-01-01 00:00:00+05:30  7.269977  7.308986  7.222101  7.296574   
1 1996-01-02 00:00:00+05:30  7.278843  7.314306  7.186638  7.239832   
2 1996-01-03 00:00:00+05:30  7.358634  7.693763  7.278843  7.294801   
3 1996-01-04 00:00:00+05:30  7.225647  7.248698  7.129896  7.227420   
4 1996-01-05 00:00:00+05:30  7.199050  7.199050  7.115711  7.177772   

      Volume  Dividends  Stock Splits  
0  104121369        0.0           0.0  
1  168743308        0.0           0.0  
2  209323879        0.0           0.0  
3  216900264        0.0           0.0  
4  166708467        0.0           0.0  


In [15]:
import pandas as pd
import numpy as np

# RSI calculation
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Backtest Strategy: RSI < 30 + 20DMA crosses above 50DMA
def backtest_strategy(df):
    df = df.copy()
    df['RSI'] = calculate_rsi(df['Close'])
    df['20DMA'] = df['Close'].rolling(window=20).mean()
    df['50DMA'] = df['Close'].rolling(window=50).mean()

    # Detect crossover: 20DMA today > 50DMA today and yesterday 20DMA <= 50DMA
    df['DMA_Cross'] = (df['20DMA'] > df['50DMA']) & (df['20DMA'].shift(1) <= df['50DMA'].shift(1))

    # Buy signal when RSI < 30 and DMA crossover happens
    df['Buy_Signal'] = (df['RSI'] < 30) & (df['DMA_Cross'])

    # Position: hold after buy (forward fill), enter position next day (shift)
    df['Position'] = 0
    df.loc[df['Buy_Signal'], 'Position'] = 1
    df['Position'] = df['Position'].replace(to_replace=0, method='ffill').shift(1).fillna(0)

    # Calculate daily returns and strategy returns
    df['Returns'] = df['Close'].pct_change()
    df['Strategy_Returns'] = df['Returns'] * df['Position']

    return df

# Calculate performance metrics
def calculate_metrics(df, risk_free_rate=0.06):  # annualized R_f
    df = df.copy()

    if df['Position'].sum() == 0:
        return {
            'Total Return': None,
            'Max Drawdown': None,
            'Annualized Return': None,
            'Annualized Volatility': None,
            'Sharpe Ratio': None,
            'Trades': 0
        }

    strategy_df = df[df['Position'] > 0].copy()

    cumulative_return = (strategy_df['Strategy_Returns'] + 1).prod() - 1
    cumulative = (strategy_df['Strategy_Returns'] + 1).cumprod()
    running_max = np.maximum.accumulate(cumulative)
    drawdown = (cumulative - running_max) / running_max
    max_drawdown = drawdown.min()

    n_days = len(strategy_df)
    annualized_return = (1 + cumulative_return) ** (252 / n_days) - 1 if n_days > 0 else None
    annualized_volatility = strategy_df['Strategy_Returns'].std() * np.sqrt(252)

    # Sharpe with risk-free rate
    excess_return = annualized_return - risk_free_rate
    sharpe_ratio = (excess_return / annualized_volatility) if annualized_volatility != 0 else None

    trades = int(df['Buy_Signal'].sum())

    return {
        'Total Return': cumulative_return,
        'Max Drawdown': max_drawdown,
        'Annualized Return': annualized_return,
        'Annualized Volatility': annualized_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Trades': trades
    }

results = {}
metrics_summary = {}

for symbol, df in stock_data.items():
    df_bt = backtest_strategy(df)
    metrics = calculate_metrics(df_bt)
    results[symbol] = df_bt
    metrics_summary[symbol] = metrics

    print(f"\nMetrics for {symbol}:")
    if metrics['Trades'] == 0:
        print("No trades triggered by the strategy in the backtest period.")
    else:
        for k, v in metrics.items():
            if k == 'Trades':
                print(f"{k}: {v}")
            elif v is None or pd.isna(v):
                print(f"{k}: N/A")
            else:
                print(f"{k}: {v:.2%}")



Metrics for RELIANCE.NS:
Total Return: 20439.05%
Max Drawdown: -77.59%
Annualized Return: 20.36%
Annualized Volatility: 75.59%
Sharpe Ratio: 18.99%
Trades: 1

Metrics for HDFCBANK.NS:
Total Return: 41357.42%
Max Drawdown: -55.11%
Annualized Return: 25.32%
Annualized Volatility: 32.19%
Sharpe Ratio: 60.01%
Trades: 2

Metrics for INFY.NS:
No trades triggered by the strategy in the backtest period.

Metrics for TATAMOTORS.NS:
Total Return: 2377.79%
Max Drawdown: -89.44%
Annualized Return: 10.40%
Annualized Volatility: 45.63%
Sharpe Ratio: 9.64%
Trades: 2

Metrics for HINDUNILVR.NS:
Total Return: 2588.12%
Max Drawdown: -42.26%
Annualized Return: 18.19%
Annualized Volatility: 26.45%
Sharpe Ratio: 46.11%
Trades: 2


/tmp/ipython-input-15-4034684323.py:31: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['Position'] = df['Position'].replace(to_replace=0, method='ffill').shift(1).fillna(0)
/tmp/ipython-input-15-4034684323.py:31: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['Position'] = df['Position'].replace(to_replace=0, method='ffill').shift(1).fillna(0)
/tmp/ipython-input-15-4034684323.py:31: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['Position'] = df['Position'].replace(to_replace=0, method='ffill').shift(1).fillna(0)
/tmp/ipython-input-15-4034684323.py:31: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['Position'] = df['Position'].replace(to_replace=0, method='ffill').shift(1).fillna(0)
/tmp/ipython-input-15-4034684323.py:31: 

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def calculate_macd(close, fast=12, slow=26, signal=9):
    exp1 = close.ewm(span=fast, adjust=False).mean()
    exp2 = close.ewm(span=slow, adjust=False).mean()
    macd = exp1 - exp2
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    macd_hist = macd - signal_line
    return macd, signal_line, macd_hist

def add_technical_indicators(df):
    df = df.copy()
    df['RSI'] = calculate_rsi(df['Close'])
    macd, macd_signal, macd_hist = calculate_macd(df['Close'])
    df['MACD'] = macd
    df['MACD_Signal'] = macd_signal
    df['MACD_Hist'] = macd_hist
    df['Volume'] = df['Volume']

    df['Next_Return'] = df['Close'].pct_change().shift(-1)
    df['Target'] = (df['Next_Return'] > 0).astype(int)
    df = df.dropna(subset=['RSI', 'MACD', 'MACD_Signal', 'MACD_Hist', 'Volume', 'Target'])
    return df


def train_predict_ml(df):
    df = add_technical_indicators(df)

    features = ['RSI', 'MACD', 'MACD_Signal', 'MACD_Hist', 'Volume']
    X = df[features]
    y = df['Target']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)  # Time series split style

    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    return accuracy, model

# --- Example usage ---

for symbol, df in stock_data.items():
    accuracy, model = train_predict_ml(df)
    print(f"ML Prediction Accuracy for {symbol}: {accuracy:.2%}")


ML Prediction Accuracy for RELIANCE.NS: 48.17%
ML Prediction Accuracy for HDFCBANK.NS: 53.41%
ML Prediction Accuracy for INFY.NS: 49.43%
ML Prediction Accuracy for TATAMOTORS.NS: 51.24%
ML Prediction Accuracy for HINDUNILVR.NS: 50.24%


In [ ]:
import gspread
from google.oauth2.service_account import Credentials
import pandas as pd

# Authenticate with Google Sheets
def init_gsheets(creds_json_path, sheet_name):
    scopes = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/drive'
    ]
    creds = Credentials.from_service_account_file(creds_json_path, scopes=scopes)
    client = gspread.authorize(creds)
    sheet = client.open(sheet_name)
    return sheet


# Prepare trade log dataframe from strategy df
def extract_trade_log(df):
    df = df.copy()
    df['Trade_Signal'] = df['Buy_Signal'].replace({False: '', True: 'BUY'})
    trade_log = df[df['Buy_Signal']].copy()
    trade_log = trade_log[['Date', 'Close', 'Trade_Signal']]
    trade_log['P&L'] = df['Strategy_Returns'].cumsum()
    return trade_log

# Calculate summary P&L and win ratio
def summary_metrics(df):
    total_pnl = df['Strategy_Returns'].sum()
    trades = df['Buy_Signal'].sum()
    wins = ((df['Strategy_Returns'] > 0) & (df['Position'] == 1)).sum()
    win_ratio = wins / trades if trades > 0 else 0
    summary = pd.DataFrame({
        'Metric': ['Total P&L', 'Total Trades', 'Winning Trades', 'Win Ratio'],
        'Value': [total_pnl, trades, wins, win_ratio]
    })
    return summary

# Upload DataFrame to specific sheet tab (worksheet), clears existing data first
# def upload_df_to_gsheet(sheet, worksheet_name, df):
#     try:
#         worksheet = sheet.worksheet(worksheet_name)
#         worksheet.clear()
#     except gspread.exceptions.WorksheetNotFound:
#         worksheet = sheet.add_worksheet(title=worksheet_name, rows="100", cols="20")

#     df = df.copy()

#     # Convert datetime columns to string
#     for col in df.columns:
#         if pd.api.types.is_datetime64_any_dtype(df[col]):
#             df[col] = df[col].dt.strftime('%Y-%m-%d')

#     # Replace problematic values
#     df = df.replace([np.inf, -np.inf], np.nan)
#     df = df.fillna("")  # or fillna(0) if numeric

#     worksheet.update([df.columns.values.tolist()] + df.values.tolist())

def upload_df_to_gsheet(sheet, worksheet_name, df):
    try:
        worksheet = sheet.worksheet(worksheet_name)
        worksheet.clear()
    except gspread.exceptions.WorksheetNotFound:
        worksheet = sheet.add_worksheet(title=worksheet_name, rows="100", cols="20")

    df = df.copy()

    # 1. Reset index to flatten DataFrame
    df.reset_index(drop=True, inplace=True)

    # 2. Fix datetime columns
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col] = df[col].dt.strftime('%Y-%m-%d')

    # 3. Remove problematic values
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.fillna("")  # For Google Sheets compatibility

    # 4. Ensure all column headers are strings
    df.columns = [str(c) if c != '' else 'Unnamed' for c in df.columns]

    # 5. Upload
    worksheet.update([df.columns.tolist()] + df.values.tolist())


# --- Usage example ---

creds_json = '/content/h2h.json'  # Replace with your credentials file path
sheet_name = 'h2h intern'  # Replace with your Google Sheet name

sheet = init_gsheets(creds_json, sheet_name)

for symbol, df in results.items():  # results from backtest_strategy step
    trade_log = extract_trade_log(df)
    summary = summary_metrics(df)

    upload_df_to_gsheet(sheet, f"{symbol}_Trade_Log", trade_log)
    upload_df_to_gsheet(sheet, f"{symbol}_Summary", summary)
    print(f"Uploading data for: {symbol}, rows: {len(df)}")



Uploading data for: RELIANCE.NS, rows: 7403
Uploading data for: HDFCBANK.NS, rows: 7406


/tmp/ipython-input-43-2787848431.py:77: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace([np.inf, -np.inf], np.nan)


Uploading data for: INFY.NS, rows: 7406
Uploading data for: TATAMOTORS.NS, rows: 8709
Uploading data for: HINDUNILVR.NS, rows: 7406


In [ ]:
def run_algo_pipeline(stock_list, creds_json_path, sheet_name):
    # 1. Init Google Sheets
    sheet = init_gsheets(creds_json_path, sheet_name)

    # 2. Loop through each stock
    for symbol in stock_list:
        print(f"\nProcessing: {symbol}")

        # Fetch historical data (update your fetch function if needed)
        df = yf.download(symbol, period="max", interval="1d")
        if df.empty:
            print(f"No data for {symbol}")
            continue
        df.reset_index(inplace=True)
        df['Symbol'] = symbol

        # Run strategy and calculate metrics
        strategy_df = backtest_strategy(df)
        metrics = calculate_metrics(strategy_df)

        # Prepare summary as single-row DataFrame
        summary = pd.DataFrame([metrics])

        # Log to Google Sheets
        upload_df_to_gsheet(sheet, f"{symbol}_Trade_Log", strategy_df)
        upload_df_to_gsheet(sheet, f"{symbol}_Summary", summary)

        print(f"Uploaded {symbol} results to Google Sheets.")


In [ ]:
stocks = ['RELIANCE.NS', 'HDFCBANK.NS', 'INFY.NS', 'TATAMOTORS.NS', 'HINDUNILVR.NS']
json_key_path = '/content/h2h.json'
sheet_name = 'h2h intern'

run_algo_pipeline(stocks, json_key_path, sheet_name)


/tmp/ipython-input-45-271681765.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="6mo", interval="1d")
[*********************100%***********************]  1 of 1 completed


Processing: RELIANCE.NS



/tmp/ipython-input-12-3471480569.py:31: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['Position'] = df['Position'].replace(to_replace=0, method='ffill').shift(1).fillna(0)
/tmp/ipython-input-45-271681765.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="6mo", interval="1d")
[*********************100%***********************]  1 of 1 completed

Uploaded RELIANCE.NS results to Google Sheets.

Processing: HDFCBANK.NS



/tmp/ipython-input-12-3471480569.py:31: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['Position'] = df['Position'].replace(to_replace=0, method='ffill').shift(1).fillna(0)
/tmp/ipython-input-45-271681765.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="6mo", interval="1d")
[*********************100%***********************]  1 of 1 completed

...

Uploaded INFY.NS results to Google Sheets.

Processing: TATAMOTORS.NS



/tmp/ipython-input-12-3471480569.py:31: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['Position'] = df['Position'].replace(to_replace=0, method='ffill').shift(1).fillna(0)
/tmp/ipython-input-45-271681765.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, period="6mo", interval="1d")
[*********************100%***********************]  1 of 1 completed

Uploaded TATAMOTORS.NS results to Google Sheets.

Processing: HINDUNILVR.NS



/tmp/ipython-input-12-3471480569.py:31: FutureWarning: The 'method' keyword in Series.replace is deprecated and will be removed in a future version.
  df['Position'] = df['Position'].replace(to_replace=0, method='ffill').shift(1).fillna(0)


Uploaded HINDUNILVR.NS results to Google Sheets.
